In [1]:
# Importing all the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import json

In [2]:
# each records collecting to the list
records = []
with open('C:\\Users\\manik\\Downloads\\kindle_reviews\\kindle_reviews.json', 'r') as file:
    for line in file:
        json_object = json.loads(line)
        records.append(json_object)

In [3]:
import psycopg2

In [4]:
# creating the connection
connection = psycopg2.connect(host='localhost',
                             user='postgres',
                             port=5439,
                             database='postgres',
                             password='ManiAnu@12345')
connection.set_session(readonly=False, autocommit=True)
cur = connection.cursor()

In [5]:
# creating the table reviews
create_table_reviews = """CREATE TABLE reviews (
    reviewerID VARCHAR(255),
    asin VARCHAR(255),
    reviewText TEXT,
    summary TEXT,
    helpful INT[],
    PRIMARY KEY (reviewerID, asin)
);"""

In [7]:
cur.execute(create_table_reviews)

In [8]:
# inserting the queries
insert_query = """
INSERT INTO reviews (reviewerID, asin, reviewText, summary, helpful)
VALUES (%s, %s, %s, %s, %s);
"""

In [11]:
# processsing the strings because some strings having special charecter thorwoing and exception while inseting
def sanitize_string(input_string):

    sanitized = input_string.replace('\x00', '')

    return sanitized


In [9]:
def sanitize_record(record):
    # Assuming record is a dictionary with multiple text fields
    for key, value in record.items():
        if isinstance(value, str):
            record[key] = sanitize_string(value)
    return record


In [12]:
sanitized_records = [sanitize_record(record) for record in records]

In [13]:
# records to be inserted
records_to_be_inserted = [(item['reviewerID'], item['asin'], item['reviewText'], item['summary'], item['helpful']) for item in sanitized_records]

In [14]:
# inserting the records to reviews database
cur.executemany(insert_query, records_to_be_inserted)
connection.commit()

In [15]:
# reviewerID- reviewverName mapper
reviewer_names = {}
for item in records:
    if "reviewerName" in list(item.keys()):
        if item["reviewerID"] in list(reviewer_names.keys()):
            continue
        else:
            reviewer_names[item["reviewerID"]] = [item["reviewerName"]]

In [23]:
# create table for the mapper
reviewers_query = """CREATE TABLE reviewers (
    reviewerID VARCHAR(255) PRIMARY KEY,
    reviewerName TEXT
);"""

In [24]:
# creation of the table
cur.execute(reviewers_query)

In [25]:
# insert query
insert_query_reviewers = """
INSERT INTO reviewers (reviewerID, reviewerName)
VALUES (%s, %s);
"""

In [19]:
# records to be inserted
records_to_be_inserted_rev = [(item, reviewer_names[item][0]) for item in reviewer_names.keys()]

In [26]:
# inserting the records
cur.executemany(insert_query_reviewers, records_to_be_inserted_rev)
connection.commit()